# Monday exercises II Solutions

## Group actions

### Exercise 1

(a) Define $2\times 2$ matrices $R$ and $S$. $R$ is supposed to rotate a vector 90 degrees ($\frac{\pi}{2}$-radians) in the counter-clockwise direction. $S$ gives the mirror image through the $x$-axis.

In [ ]:
import numpy as np

In [ ]:
# first column; what the matrix will do to [1,0]. Second column: what it will do to [0,1]
R = np.array([[0,-1],[1,0]])
S = np.array([[1,0],[0,-1]])


(b) Generate a random set in $\mathbb{R}^2$  of 100 points so that you can show the effect of multiplying these points with $R$ and $S$ before and after. Multiplication here refers to multiplication of a matrix and a vector. The output should not look similar for $R$ and $S$. Plot the original points and their image after applying $R$ and $S$ in the same plot. What do you want to avoid in order for them to look different?

In [ ]:
# We cannot have something  that is not rotationally invariant and not invariant under reflection
# with the x-axis
# If we generate random vectors in with all values in [0,1] x [0,1], then we 
np.random.seed(0)
pts1 =np.random.rand(2,100)
Rpts1 = R @ pts1
Spts1 = S @ pts1

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(x=pts1[0,:],y=pts1[1,:])
plt.scatter(x=Rpts1[0,:], y=Rpts1[1,:])
plt.scatter(x=Spts1[0,:], y=Spts1[1,:])
plt.legend(["Original points", "Rotation by R","Reflection by S"])

plt.show()


### Exercise 2

In this exercise, we are again trying to understand groups using matrix multiplication.

(a) Write a function that does the following:
* Takes as input a list $List$ of square $2\times 2$ matrices.
* Make a new list $List1$ of matrices that which includes $List0$ but adjoins all possible products by matrix multiplication from $List0$. Do not add matrices that are already in the list.
* Continue this procedure until either there are no new matrices or until the list has reached 100 elements (or some other upper threshold that you might prefer).


In [ ]:
# 2a. Defining a funciton to generate all possible matrices

def GenerateMat(List,n=100):
    NewMat = True
    List1 = List
    
    while NewMat:
        NewMat = False
        List0 = List1
        l = len(List1)
        
        for i in range(0,l):
            for j in range(0,l):
                A = List0[i] @ List0[j]
                
                k=0
                Notequal = True
                
                while Notequal and k < len(List1):
                    Notequal = not np.array_equal(A,List1[k])
                    k+=1
                
                if Notequal:
                    List1.append(A)
        
        if len(List1) < n:
            if len(List1) > l:
                NewMat = True
        else:
            print('Maximal length of list reached')
    
    
    return List1


(b) Test your function by implementing it with the list with $R$ and $S$. What is the output $G$? Is $G$ a group?

In [ ]:
# 2b We generate it from R and S
G = GenerateMat([R,S])
print(G)
# This contains 8 elements and it is a group

(c) For $G$ as in 2(b), update your point cloud from 1(b) so that it will be different under each group element in $G$.

In [ ]:
# 2c Our original point clould would be the same for mirroring in the x-axis and rotation by -90 degrees
# So we change it.

# Option 1: Make it into a triangle
# We move all the points into a triangle bound by the lines x=0, y=2x, y= 2-2x
# Recall that pts1 is on uniform on [0,1]x[0,1].
# For each y, we map the x-coordinate from [0,1] to [y/2,1-y/2]
# hence, we map x to 0.5y +(1-y)x

pts2 = np.array([0.5*pts1[1] + (1-pts1[1])*pts1[0],pts1[1]])
plt.scatter(x=pts2[0,:],y=pts2[1,:])
plt.xlim(0, 1)

plt.show()


In [ ]:
# Plot all elements in G
for k in range(0,len(G)):
    A = G[k]
    ptsG = A @ pts2
    plt.scatter(x=ptsG[0,:],y=ptsG[1,:])

plt.show()


In [ ]:
# Option 2: We introduce some asymmetry in the point block
pts3 = np.array([2*(pts1[0]**(1/4)),pts1[1]])
plt.scatter(x=pts3[0,:],y=pts3[1,:])

plt.show()

In [ ]:
# Plot all elements in G
for k in range(0,len(G)):
    A = G[k]
    ptsG = A @ pts3
    plt.scatter(x=ptsG[0,:],y=ptsG[1,:])

plt.show()


(d) Test your function with the list only containing the matrix
$$P = \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix}.$$
Check that this is not a group.

If the process stops withing the time-limit, how can we not end up with a group? Can you modify the code to check if it is possible, and give an error if this is not the case.

In [ ]:
# 2d We see on the other hand that
P = np.array([[1,0],[0,0]])
print(GenerateMat([P]))
# which is not a group.

In [ ]:
# We will never get a group if our list contains non-invertible elements.
# Luckily, if our initial matrices are invertible, so will all of their products.


def GenerateGroup(List,n=100):
    NewMat = True
    List1 = List
    
    k=0
    while NewMat and (k < len(List1)):
        if np.linalg.det(List1[k]) == 0:
            print("Error: Non invertible elements in the list")
            NewMat = False
        else:
            k +=1
            
    
    while NewMat:
        NewMat = False
        List0 = List1
        l = len(List1)
        
        for i in range(0,l):
            for j in range(0,l):
                A = List0[i] @ List0[j]
                
                k=0
                Notequal = True
                
                while Notequal and k < len(List1):
                    Notequal = not np.array_equal(A,List1[k])
                    k+=1
                
                if Notequal:
                    List1.append(A)
        
        if len(List1) < n:
            if len(List1) > l:
                NewMat = True
        else:
            print("Maximal length of list reached")
    
    
    return List1


## Rotation and reflection group

(a) Recall the definition of the rotation matrix $R$ ($90^{\circ}$ rotation) and mirroring $S$ along the $x$-axis.
Define the group $D_4$ as pairs of tuple $(r,s)$, $r=0,1,2,3$, $s=0,1$ such that
$$(r_1, s_1) \cdot (r_2, s_2) = (r_3,s_3) \qquad \text{if} \qquad R^{r_1} S^{s_1} R^{r_2} S^{s_2} = R^{r_3} S^{s_3}.$$
Show that
$$(r_1 ,s_1) \cdot (r_2, s_2) = (r_1 +(-1)^{s_1} r_2 \bmod 4, s_1+s_2 \bmod 2).$$
What is the identity? Find the formula for the inverse.

In [ ]:
# We observe that
print('SR', R @ S,'\n', 'S R^3 = SR^{-1}', S@(np.linalg.matrix_power(R,3)))

Using this property iteratively and using that $S^0 =I$ is the identity matrix,
$$S^{s} R^r  =R^{(-1)^s r} S^s$$
This gives us
$$R^{r_1} S^{s_1} R^{r_2} S^{s_2} = R^{r_1} R^{(-1)^{s_2} r_2} S^{s_1}  S^{s_2} = R^{r_1+(-1)^{s_1}r_2}S^{s_1+s_2}$$
Since $R^4 =S^2 = I$, the result follows.

The identity is $(0,0)$. For a tuple $(r,0)$, we see that $(- r \bmod 4, 0)$ is the inverse. For $(r,1)$, the inverse is $(r,1)$. We can summarize this as
$$(r,s)^{-1} = ((-1)^{s+1} r \bmod 4, s).$$

(b) Make a class D4 representing the group, which as a defined method defined for product and inverse.

In [ ]:
class D4:
    def __init__(self, element):
        # Valid elements of D4
        self.elements = tuples_list = [(r, s) for r in range(4) for s in range(2)]
        
        if element not in self.elements:
            raise ValueError("Invalid element for D4")
        
        self.element = element
    
    def __repr__(self):
        return f"D4({self.element})"
    
    def __mul__(self, other):
        if not isinstance(other, D4):
            raise ValueError("Can only multiply two D4 elements")
        
        result = self.multiply(self.element, other.element)
        return D4(result)
    
    def multiply(self, a, b):
        c0 = a[0]+(-1)**b[1]*b[0]
        c1 = a[1]+b[1]
        
        c0 = c0%4
        c1 = c1%2
        
        return (c0,c1)
    
    def mult(self, b):
        a = self.element
        c0 = a[0]+(-1)**b[1]*b[0]
        c1 = a[1]+b[1]
        
        c0 = c0%4
        c1 = c1%2
        
        return (c0,c1)
    
    
    
    def inverse(self):
        a= self.element
        c0 = (-1)**a[1]*a[0]
        c0 = c0%4
        c1 = a[1]
        
        return (c0,c1)


In [ ]:
a = D4((2,1))
b = D4((3,1))
print(a*b)

In [ ]:
a.inverse()

(c) Make a function $f(g,img)$, where $g$ is in your new class `D4` and $img$ is an image and where the output is the action of $D4$ on the image. You can use standard functions in `cv2` for mirroring and translations. Test it on an image, showing all the configurations.

In [ ]:
import cv2

In [ ]:
def Action(a,image):
    tup= a.element
    r = tup[0]
    s = tup[1]
    
    img = image
    
    if s ==1:
        img = cv2.flip(img,0)
        
    if r ==1:
        img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif r==2:
        img = cv2.rotate(img, cv2.ROTATE_180)
    elif r==3:
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    
    
    return img
        

In [ ]:
fig, axs = plt.subplots(4, 2)
I = cv2.imread("../../lecture_notes/barbara.bmp",cv2.IMREAD_GRAYSCALE).astype(np.float32)

for r in range(4):
    for s in range(2):
        aD4 = D4((r,s))
        axs[r,s].imshow(Action(aD4,I), cmap="gray")


plt.tight_layout()

plt.show